# The `QuantumState` class

Express an N-qubit quantum state as an array of N columns, where rows are binary bit values, complemented by a vector of coefficients. Note the similarity with the symplectic representation - indeed, the QuantumState class will infact set the above array as the X block in a PauliwordOp, with the Z block its complement.

What we are doing here is writing $|0\rangle = Z|0\rangle$ and $|1\rangle = X|0\rangle$, which ensures correct phases when multiplying the state by Pauli operators, since

$$X|0\rangle = XZ|0\rangle = -iY|0\rangle = |1\rangle,\; X|1\rangle = XX|0\rangle = |0\rangle$$
$$Y|0\rangle = YZ|0\rangle = iX|0\rangle = i|1\rangle,\; Y|1\rangle = YX|0\rangle = -iZ|0\rangle = -i|0\rangle$$
$$Z|0\rangle = ZZ|0\rangle = |0\rangle,\; Z|1\rangle = ZX|0\rangle = iY|0\rangle = -|1\rangle$$

Finally, we have $$| \vec{b} \rangle = \bigotimes_{i=1}^N \big(b_i X + (1-b_i) Z\big) | \vec{0} \rangle$$ and we may drop the zero vector and use the functionality of PauliwordOp to manipulate quantum states. In this represenation, a quantum state is stored as an operator consisting of Paulis $X, Z$, which is implicitly applied to the zero (or vacuum) state. 

In [1]:
import numpy as np
from symmer.utils import unit_n_sphere_cartesian_coords
from symmer.symplectic import PauliwordOp, QuantumState, array_to_QuantumState

Firstly, note the correct phases under multiplication by a Pauli Y:

In [2]:
zero = QuantumState([[0]])
one = QuantumState([[1]])
Y = PauliwordOp(['Y'], [1])

print(f'{zero} -> {Y * zero}')
print(f'{one} -> {Y * one}')

 1.000 |0> ->  0.000+1.000j |1>
 1.000 |1> ->  0.000-1.000j |0>


Now, let's see what happens when we apply the Hadamard gate to the zero state:

In [3]:
psi = QuantumState([[0,0]])
had = PauliwordOp(['XZ','ZX','XX', 'ZZ'], np.ones(4)/2) # 2-qubit Hadamard gate decomposed over Paulis
eq_superposition = had * psi
print(f'Zero state: {psi}\n')
print(f'After application of the Hadamard gate:\n\n{eq_superposition}')

Zero state:  1.000 |00>

After application of the Hadamard gate:

 0.500+0.000j |00> +
 0.500+0.000j |01> +
 0.500+0.000j |10> +
 0.500-0.000j |11>


Observe that the QuantumState is represented internally by its `state_op`, a PauliwordOp object that governs its behaviour under multiplication

In [4]:
print(eq_superposition.state_op)

 0.500+0.000j ZZ +
 0.500+0.000j ZX +
 0.500+0.000j XZ +
 0.500-0.000j XX


Try evaluating expectation values for randomly generated state and Hermitian operators:

In [5]:
def random_state(num_qubits, num_terms):
    """ Generates a random normalized QuantumState
    """
    # random binary array with N columns, M rows
    random_state = np.random.randint(0,2,(num_terms,num_qubits))
    # random vector of coefficients
    coeff_vec = (
        np.random.rand(num_terms) + 
        np.random.rand(num_terms)*1j
    )
    return QuantumState(random_state, coeff_vec).normalize

def random_herm_op(num_qubits, num_terms):
    """ Generates a random PaulwordOp
    """
    # random binary array with 2N columns, M rows
    random_symp_mat = np.random.randint(0,2,(num_terms,num_qubits*2))
    # random vector of coefficients
    coeff_vec = np.random.rand(num_terms)
    coeff_vec/=coeff_vec[::-1]
    coeff_vec*=(2*np.random.randint(0,2,num_terms)-1)
    return PauliwordOp(random_symp_mat, coeff_vec)

In [6]:
N = 10 # number of qubits
M = 10 # number of terms

psi_1 = random_state(N, M)
psi_2 = random_state(N, M)

print(psi_1)
print()
print(psi_2)

 0.338+0.266j |1001110001> +
 0.120+0.197j |0101010100> +
 0.334+0.109j |1100110011> +
 0.036+0.118j |0000101101> +
 0.052+0.151j |0001101100> +
 0.275+0.223j |1101001000> +
 0.185+0.044j |0101010101> +
 0.079+0.353j |1010010100> +
 0.357+0.015j |1101111101> +
 0.290+0.306j |0101011100>

 0.080+0.197j |1011111011> +
 0.311+0.367j |0111111100> +
 0.299+0.324j |0001100110> +
 0.019+0.060j |1101011011> +
 0.402+0.173j |1110001110> +
 0.002+0.072j |0010010111> +
 0.066+0.288j |0000001110> +
 0.030+0.198j |1101101110> +
 0.167+0.310j |0100010011> +
 0.086+0.263j |0010110011>


In [7]:
psi = (psi_1 + psi_2).normalize
print(psi)
print()
print('Norm:', psi.conjugate * psi)

 0.047+0.204j |0000001110> +
 0.026+0.083j |0000101101> +
 0.212+0.229j |0001100110> +
 0.037+0.107j |0001101100> +
 0.001+0.051j |0010010111> +
 0.061+0.186j |0010110011> +
 0.118+0.219j |0100010011> +
 0.085+0.140j |0101010100> +
 0.131+0.031j |0101010101> +
 0.205+0.216j |0101011100> +
 0.220+0.259j |0111111100> +
 0.239+0.188j |1001110001> +
 0.056+0.250j |1010010100> +
 0.057+0.139j |1011111011> +
 0.236+0.077j |1100110011> +
 0.195+0.158j |1101001000> +
 0.013+0.042j |1101011011> +
 0.021+0.140j |1101101110> +
 0.252+0.010j |1101111101> +
 0.284+0.122j |1110001110>

Norm: (1+0j)


In [8]:
op = random_herm_op(N, 10)
print(op)
print()
print('Expectation value <psi|op|psi> =', psi.conjugate * op * psi)

-0.450+0.000j ZXYIIYYZIZ +
-6.210+0.000j IZYXYZYZXX +
-33.687+0.000j YIZIYYYXYI +
-2.248+0.000j XXZZYIYXXX +
-3.166+0.000j YXZXXZIZYY +
-0.316+0.000j ZYXIYIIZYI +
-0.445+0.000j XYYXXYZYXX +
 0.030+0.000j IYXIIIZYZY +
-0.161+0.000j YIXZIIYZZI +
-2.221+0.000j YIZXYIIYII

Expectation value <psi|op|psi> = 0


It is also possible to convert from a $2^N$ dimensional statevector over $N$ qubits to a `QuantumState` object, via the `array_to_QuantumState` function:

In [9]:
statevector = np.zeros(2**10)
statevector[0]   = np.sqrt(1/6)
statevector[250] = np.sqrt(2/6)
statevector[500] = np.sqrt(3/6)
psi = array_to_QuantumState(statevector)
print(psi)
print(psi.conjugate * psi)

 0.408 |0000000000> +
 0.577 |0011111010> +
 0.707 |0111110100>
1.0
